In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("./archivos-bd/data_uci.csv", delimiter=";")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data["riesgo"].value_counts()

# plt.bar(data["riesgo"].unique(), data["riesgo"].value_counts())

sns.countplot(x=data["riesgo"])

In [ ]:
data.columns

In [ ]:
corr = data.iloc[:, :5].corr()

# Grafica con Seaborn de la matriz de correlación
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))
plt.tight_layout()

In [ ]:
sns.pairplot(data)

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

print("Edad:",data["edad"].unique())

print("Dolor toraxico:", data["td"].unique())

print("Presion arterial reposo:",data["par"].unique())

print("Colesterol", data["col"].unique())

print("Frecuencia cardiaca maxima:", data["fcm"].unique())

print("Riesgo:", data["riesgo"].unique())



In [ ]:
data[data["fcm"] == 0]

In [ ]:
data[data["par"] == -1]

In [ ]:
mediaFcm = int(data[data["fcm"] != 0]["fcm"].mean())

data["fcm"] = data["fcm"].replace(0, mediaFcm)

print(data[data["fcm"] == 0])

data["fcm"].unique()

In [ ]:
mediaFcm = int(data[data["par"] != -1]["par"].mean())

data["par"] = data["par"].replace(-1, mediaFcm)

print(data[data["par"] == -1])

data["par"].unique()

In [ ]:
sns.boxplot(data)

In [ ]:
data_sin_riesgo = data.iloc[:, :5]

normalized_df=(data_sin_riesgo-data_sin_riesgo.min())/(data_sin_riesgo.max()-data_sin_riesgo.min())

sns.boxplot(normalized_df)

### Observacion

Encontramos que hay un valor de colesterol que esta completametne elejado de los valores posibles ya que es más del doble del nivel de colesterol aceptado.
El procesamiendo que vamos a realizar es reemplazar el valor por el valor de colesterol medio porque pensamos que este valor tiene un error de medicion.


In [ ]:
data[data["col"] >= 500]

In [ ]:
mediaCol = int(data[data["col"] != 564]["col"].mean())

data["col"] = data["col"].replace(564, mediaCol)

print(data[data["col"] == 564])

data_sin_riesgo = data.iloc[:, :5]

normalized_df=(data_sin_riesgo-data_sin_riesgo.min())/(data_sin_riesgo.max()-data_sin_riesgo.min())

sns.boxplot(normalized_df)

In [ ]:
normalized_df

In [ ]:
data["riesgo"] = data["riesgo"].map({'alto':1,'bajo':0})
data

# Entrenamiento del modelo


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [ ]:
x = normalized_df.copy()
y = data["riesgo"].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=1)

print(x.shape, x_train.shape, x_test.shape)
print(y.shape, y_train.shape, y_test.shape)

## Regresión Logística

In [ ]:
# Instanciar un modelo de Regresión Logística
model_log_reg = LogisticRegression(verbose=2)

# Entrenar el modelo con el conjunto de entrenamiento
model_log_reg.fit(x_train, y_train)

# Calculando la precisión para el conjunto de entrenamiento
x_train_prediction = model_log_reg.predict(x_train)
train_accuracy = accuracy_score(x_train_prediction, y_train)
train_f1 = f1_score(x_train_prediction, y_train)
print('Accuracy para los datos de entrenamiento : ', train_accuracy)
print('F1-Score para los datos de entrenamiento : ', train_f1)

# Calculando la precisión para el conjunto de test
x_test_prediction = model_log_reg.predict(x_test)
test_accuracy = accuracy_score(x_test_prediction, y_test)
test_f1 = f1_score(x_test_prediction, y_test)
print('Accuracy para los datos de test : ', test_accuracy)
print('F1-Score para los datos de test : ', test_f1)

## Regresión Logística con regularización Lasso

In [ ]:
# Instanciar un modelo de Regresión Logística
model_log_reg = LogisticRegression(penalty="l1", C=0.2, verbose=2, solver="liblinear")

# Entrenar el modelo con el conjunto de entrenamiento
model_log_reg.fit(x_train, y_train)

# Calculando la precisión para el conjunto de entrenamiento
x_train_prediction = model_log_reg.predict(x_train)
train_accuracy = accuracy_score(x_train_prediction, y_train)
train_recall = recall_score(x_train_prediction, y_train)
train_f1 = f1_score(x_train_prediction, y_train)
print('Accuracy para los datos de entrenamiento : ', train_accuracy)
print('Recall para los datos de entrenamiento : ', train_recall)
print('F1-Score para los datos de entrenamiento : ', train_f1)

# Calculando la precisión para el conjunto de test
x_test_prediction = model_log_reg.predict(x_test)
test_accuracy = accuracy_score(x_test_prediction, y_test)
test_recall = recall_score(x_test_prediction, y_test)
test_f1 = f1_score(x_test_prediction, y_test)
print('Accuracy para los datos de test : ', test_accuracy)
print('Recall para los datos de entrenamiento : ', test_recall)
print('F1-Score para los datos de test : ', test_f1)

## Naive-Bayes

In [ ]:
# Instanciamos un modelo Naive Bayes
model_NB = GaussianNB()

# Entrenar el modelo con el conjunto de entrenamiento
model_NB.fit(x_train, y_train)

# Calculando la precisión para el conjunto de entrenamiento
x_train_prediction = model_NB.predict(x_train)
train_accuracy = accuracy_score(x_train_prediction, y_train)
train_f1 = f1_score(x_train_prediction, y_train)
print('Accuracy para los datos de entrenamiento : ', train_accuracy)
print('F1-Score para los datos de entrenamiento : ', train_f1)

# Calculando la precisión para el conjunto de test
x_test_prediction = model_NB.predict(x_test)
test_accuracy = accuracy_score(x_test_prediction, y_test)
test_f1 = f1_score(x_test_prediction, y_test)
print('Accuracy para los datos de test : ', test_accuracy)
print('F1-Score para los datos de test : ', test_f1)

## K-Nearest-Neighbors

In [ ]:
k_range = [i for i in range(1, 21)]
param_grid = dict(n_neighbors=k_range)

grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=7, scoring="recall")

grid_search = grid.fit(x_train, y_train)

print("Score en el conjunto de tests: {:.2f}".format(grid_search.score(x_test, y_test)))
print("Mejores parámetros: {}".format(grid_search.best_params_))
print("Mejor Score de validacion Cruzada: {:.2f}".format(grid_search.best_score_))

In [ ]:
# Instanciamos un modelo KNN
model_KNN = KNeighborsClassifier(grid_search.best_params_["n_neighbors"])

# Entrenar el modelo con el conjunto de entrenamiento
model_KNN.fit(x_train, y_train)

# Calculando la precisión para el conjunto de entrenamiento
x_train_prediction = model_KNN.predict(x_train)
train_accuracy = accuracy_score(x_train_prediction, y_train)
train_f1 = f1_score(x_train_prediction, y_train)
print('Accuracy para los datos de entrenamiento : ', train_accuracy)
print('F1-Score para los datos de entrenamiento : ', train_f1)

# Calculando la precisión para el conjunto de test
x_test_prediction = model_KNN.predict(x_test)
test_accuracy = accuracy_score(x_test_prediction, y_test)
test_f1 = f1_score(x_test_prediction, y_test)
print('Accuracy para los datos de test : ', test_accuracy)
print('F1-Score para los datos de test : ', test_f1)